In [2]:
import numpy as np
import pandas as pd
import glob 
import os
import datetime
pd.options.display.max_columns=50

In [3]:
# results = glob.glob('./results/query_indices/*')
# list1, list2 = zip(*sorted(zip([os.stat(result).st_size for result in results], results))) 
# df = pd.read_csv(list2[-15], skiprows = 1)

In [4]:
df = pd.read_csv('data/synthetic-imports-declarations.csv')
df

,sgd.id,sgd.date,importer.id,declarant.id,country,office.id,tariff.code,quantity,gross.weight,fob.value,cif.value,total.taxes,illicit,revenue
0,SGD1,13-01-02,IMP826164,DEC3207,CNTRY680,OFFICE92,8703241128,1581,26494,2390,2809,647,0,0
1,SGD2,13-01-02,IMP837219,DEC1525,CNTRY680,OFFICE51,8703232926,1,3910,204098,266140,3262,0,0
2,SGD3,13-01-02,IMP117406,DEC4146,CNTRY680,OFFICE59,8517180000,1,699231,302275,302275,5612,0,0
3,SGD4,13-01-02,IMP435108,DEC4242,CNTRY376,OFFICE51,8703222900,1288,22958,3019,4160,514,0,0
4,SGD5,13-01-02,IMP717900,DEC6324,CNTRY454,OFFICE92,8545200000,42,21248,156348,239549,397,1,980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,SGD99996,13-12-31,IMP435297,DEC4557,CNTRY376,OFFICE40,8703222900,18533,37558,2319,2621,2731,0,0
99996,SGD99997,13-12-31,IMP591216,DEC1174,CNTRY759,OFFICE60,8517180000,1,1821,1746,2226,3355,0,0
99997,SGD99998,13-12-31,IMP398817,DEC6384,CNTRY881,OFFICE87,3911900000,2837,8679,30706,35214,2445,0,0
99998,SGD99999,13-12-31,IMP418859,DEC6689,CNTRY759,OFFICE51,8703231119,461,41359,2413,4480,546,0,0


### Results of Hybrid Experiments

In [5]:
run_id_saver = [('hybrid', 1602332296)]
results = {}
num_weeks = 2
for strategy, run_id in run_id_saver:
    results[run_id] = {}
    for week in range(num_weeks):
        days = pd.read_csv(f'results/query_indices/{run_id}-{strategy}-5.0-scratch-week-{week}.csv').columns
        start_day = datetime.date(int(days[1][:4]), int(days[1][5:7]), int(days[1][8:10])).strftime('%y-%m-%d')
        end_day = datetime.date(int(days[2][:4]), int(days[2][5:7]), int(days[2][8:10])).strftime('%y-%m-%d')
        indices = pd.read_csv('results/query_indices/1602332754-hybrid-5.0-scratch-week-0.csv', skiprows = 1)
        alldata = df[(df['sgd.date'] < end_day) & (df['sgd.date'] >= start_day)].loc[:, ['illicit', 'revenue']]
        for i, samp in enumerate(indices.columns):
            if week == 0:
                results[run_id][f'{samp}-pre'] = []
                results[run_id][f'{samp}-rec'] = []
                results[run_id][f'{samp}-rev'] = []
            ind = indices.iloc[:,0].dropna().values.astype(int)
            chosen = df.iloc[ind].loc[:, ['illicit', 'revenue']]
            # Recall and revenue
            if chosen.empty:
                pre = rec = rev = 0
            else:
                pre = sum(chosen['illicit'])/chosen['illicit'].count()
                rec = sum(chosen['illicit'])/sum(alldata['illicit'])
                rev = sum(chosen['revenue'])/sum(alldata['revenue'])
            print(f'Week {week}, subsampler {samp}: Precision {pre} Recall {rec}, Revenue {rev}')
            results[run_id][f'{samp}-pre'].append(pre)
            results[run_id][f'{samp}-rec'].append(rec)
            results[run_id][f'{samp}-rev'].append(rev)

Week 0, subsampler random: Precision 0.06976744186046512 Recall 0.00473186119873817, Revenue 0.006639184733070688
Week 0, subsampler badge: Precision 0.06976744186046512 Recall 0.00473186119873817, Revenue 0.006639184733070688
Week 1, subsampler random: Precision 0.06976744186046512 Recall 0.005008347245409015, Revenue 0.006474292769210679
Week 1, subsampler badge: Precision 0.06976744186046512 Recall 0.005008347245409015, Revenue 0.006474292769210679


In [6]:
run_id = 1602332296
pd.DataFrame.from_dict(results[run_id])

,random-pre,random-rec,random-rev,badge-pre,badge-rec,badge-rev
0,0.069767,0.004732,0.006639,0.069767,0.004732,0.006639
1,0.069767,0.005008,0.006474,0.069767,0.005008,0.006474


# Result for novelty

In [8]:
run_id_saver = [('hybrid', 1602332296)]
novelty = {}
num_weeks = 2
old_IID = set()
for strategy, run_id in run_id_saver:
    novelty[run_id] = {}
    for week in range(num_weeks):
        days = pd.read_csv(f'results/query_indices/{run_id}-{strategy}-5.0-scratch-week-{week}.csv').columns
        start_day = datetime.date(int(days[1][:4]), int(days[1][5:7]), int(days[1][8:10])).strftime('%y-%m-%d')
        end_day = datetime.date(int(days[2][:4]), int(days[2][5:7]), int(days[2][8:10])).strftime('%y-%m-%d')
        indices = pd.read_csv('results/query_indices/1602332754-hybrid-5.0-scratch-week-0.csv', skiprows = 1)
        alldata = df[(df['sgd.date'] < end_day) & (df['sgd.date'] >= start_day)].loc[:, ['illicit', 'revenue', 'importer.id']]
        alldata = alldata[~alldata['importer.id'].isin(old_IID)]
        if alldata.empty:
            continue
        for i, samp in enumerate(indices.columns):
            if week == 0:
                novelty[run_id][f'{samp}-pre'] = []
                novelty[run_id][f'{samp}-rec'] = []
                novelty[run_id][f'{samp}-rev'] = []
            ind = indices.iloc[:,0].dropna().values.astype(int)
            chosen = df.iloc[ind]
            chosen = chosen[~chosen['importer.id'].isin(old_IID)]
            if chosen.empty:
                pre = rec = rev = 0
            else:
                pre = sum(chosen['illicit'])/chosen['illicit'].count()
                rec = sum(chosen['illicit'])/sum(alldata['illicit'])
                rev = sum(chosen['revenue'])/sum(alldata['revenue'])
            print(f'Week {week}, subsampler {samp}: Precision {pre} Recall {rec}, Revenue {rev}')
            novelty[run_id][f'{samp}-pre'].append(pre)
            novelty[run_id][f'{samp}-rec'].append(rec)
            novelty[run_id][f'{samp}-rev'].append(rev)
        old_IID = old_IID.union(set(alldata['importer.id'].values))

Week 0, subsampler random: Precision 0.06976744186046512 Recall 0.00473186119873817, Revenue 0.006639184733070688
Week 0, subsampler badge: Precision 0.06976744186046512 Recall 0.00473186119873817, Revenue 0.006639184733070688
Week 1, subsampler random: Precision 0 Recall 0, Revenue 0
Week 1, subsampler badge: Precision 0 Recall 0, Revenue 0


In [9]:
run_id = 1602332296
pd.DataFrame.from_dict(novelty[run_id])

,random-pre,random-rec,random-rev,badge-pre,badge-rec,badge-rev
0,0.069767,0.004732,0.006639,0.069767,0.004732,0.006639
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
